In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2, DenseNet121, VGG16, InceptionV3, MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Optimizer, Adam,RMSprop, SGD
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#import argparse
import cv2
import os
import shutil
import math
import gc
from datetime import datetime

In [ ]:
# Class for the optimizer Itao ( a new Iterative thresholding algorithm based optimizer)
class Itao(Optimizer):
    def __init__(self, k=10.0, lamda=0.00001, name="Itao", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("k", k) # tuning hyperparameter K
        self._set_hyper("lamda", lamda) # Tikhonov parameter lamda
        
    

    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        
        var_dtype = var.dtype.base_dtype
        k = self._get_hyper("k", var_dtype)
        lamda = self._get_hyper("lamda", var_dtype)
        new_var = (k/(k+lamda))*var - grad/(k+lamda) # update variables
        var.assign(new_var)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "k": self._serialize_hyperparameter(self._k),
            "lamda": self._serialize_hyperparameter(self._lamda),
            
        }
    def get_config(self):
        config = super(Itao, self).get_config()
        config.update({
            "k": self._serialize_hyperparameter("k"),
            "lamda": self._serialize_hyperparameter("lamda"),
            
        })
        return config

In [ ]:
# Loading data and labels (the result of "preparationOfDataset.ipynb")
data_train = np.load('../new_3000_data_train_224.npy') 
labels_train = np.load('../new_3000_labels_train_224.npy')

In [ ]:
data_train.shape

In [ ]:
#Shuffling Data
np.random.seed(12345)
indx=np.arange(data_train.shape[0])          
np.random.shuffle(indx)
data_train = data_train[indx]
labels_train = labels_train[indx]

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
  )

In [ ]:
# Function to create a model based one of these models: MobileNetV2, ResNet50V2, VGG16 or DenseNet121
def create_model(base_model):
    lam = l2(1e-5) # L2 regularization
    if base_model == "MobileNetV2":
        baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    elif base_model == "ResNet50V2":
        baseModel = ResNet50V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    elif base_model == "VGG16":
        baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    else:
        baseModel = DenseNet121(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3))) 
            
    for layer in baseModel.layers:
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', lam)
    x = baseModel.output
    x = Flatten(name="flatten")(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu', kernel_regularizer=lam)(x)
    x = Dropout(0.1)(x)
    x = Dense(3, activation="softmax", kernel_regularizer=lam)(x) 

    model = Model(inputs=baseModel.input, outputs=x)
    for layer in baseModel.layers: 
        layer.trainable =  True #False
    
    return model

In [ ]:
# To see trainable parameters
#model = create_model()
#model.summary()

In [ ]:
labels = np.argmax(labels_train, axis=1)

In [ ]:
# Import function to plot accuracy and loss curves during training and validation phases
# Import functions to produce evaluation metrics and plot the confusion matrix
from utils import plot_acc_loss
from utils import evaluation_metrics

In [ ]:
# Cross-validation object
from sklearn.model_selection import KFold, StratifiedKFold
skf = StratifiedKFold(n_splits =5, random_state = 7, shuffle = True)

In [ ]:
# Create a directory to save results and models of different folds (5 fold cross-validation)
!mkdir covid19_models_cross_val_MobileNetV2
!mkdir covid19_models_cross_val_ResNet5OV2
!mkdir covid19_models_cross_val_VGG16
!mkdir covid19_models_cross_val_DenseNetV2

In [ ]:
validation_accuracy = []
validation_loss = []

# Select base model as feature extractor
base_model = "MobileNetV2"
#base_model = "ResNet50V2"
#base_model = "VGG16"
#base_model = "DenseNet121"
save_dir = "covid19_models_cross_val_33" + base_model + "/"
fold_var = 1

for train_index, val_index in skf.split(np.zeros(len(data_train)),labels):
  
    training_data = data_train[train_index]
    training_labels = labels_train[train_index]
    validation_data = data_train[val_index]
    validation_labels = labels_train[val_index]

    model = create_model(base_model)
    opt = Itao() 
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])
    
    best_model_path = save_dir + 'model_' + base_model + '_fold_' + str(fold_var) + '.h5'
    mcp_save = ModelCheckpoint(best_model_path, save_best_only=True, monitor='val_acc', mode='max', verbose=1)
    callbacks = [ mcp_save]
    
    print("[INFO] training K-Fold...step: " + str(fold_var))
    
    histories = []
    lesK = [ 40.0, 80.0, 120.0] # Scheduling of hyperparameter K during epochs
    lesEpochs = [40, 40, 20]
    BS_tr=32
    BS_val = 32
    for i in range(len(lesK)):
        opt.k =lesK[i]
        print("[INFO] Training model...K = " + str(lesK[i]))
        
        H = model.fit(
            trainAug.flow(training_data, training_labels, batch_size=BS_tr),
            #steps_per_epoch=len(training_data) // BS_tr ,
            validation_data=(validation_data, validation_labels),
            #validation_steps=len(validation_data) // BS_val ,
            epochs=lesEpochs[i], callbacks=callbacks,
            shuffle=True)
        histories.append(H.history)
    
    #np.save(save_dir + 'history_' + base_model + '-' + str(fold_var) + '.npy', Hs.history)
    dict_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}
    for hist in histories:
        for key , value in dict_history.items():
            for a in hist[key]:
                dict_history[key].append(a )
    
    plot_acc_loss(dict_history, np.sum(lesEpochs))

    #Loading best model
    model.load_weights(best_model_path)
    #Saving weights
    model.save_weights(save_dir + 'model_' + base_model + '_fold_' + str(fold_var) + '_weights.h5')
    results = model.evaluate(validation_data, validation_labels)
    results = dict(zip(model.metrics_names,results))
    
    validation_accuracy.append(results['acc'])
    validation_loss.append(results['loss'])
                       
    plt.figure()
    plt.show()

    evaluation_metrics(model, validation_data, validation_labels, fold_var, BS_val)
    
    #Saving accuracy and loss curves
    np.save(save_dir + base_model + "_fold_" + str(fold_var) + "_acc.npy", dict_history["acc"])
    np.save(save_dir + base_model + "_fold_" + str(fold_var) + "_loss.npy", dict_history["loss"])
    np.save(save_dir + base_model + "_fold_" + str(fold_var) + "_val_acc.npy", dict_history["val_acc"])
    np.save(save_dir + base_model + "_fold_" + str(fold_var) + "_val_loss.npy", dict_history["val_loss"])    

    tf.keras.backend.clear_session()
    
    fold_var = fold_var + 1 # Next fold

In [ ]:
print("Average accuracy : ", np.mean(validation_accuracy))
print("Average loss : ", np.mean(validation_loss))